In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/drone-bird-classification/drone_or_bird/birds/bbb3eea4-f36f-4ed4-ac21-7800c4f93f7a.jpg
/kaggle/input/drone-bird-classification/drone_or_bird/birds/ae8d10f5-719e-4c9c-9932-73d05d7ccff6.jpg
/kaggle/input/drone-bird-classification/drone_or_bird/birds/dd7fe0e8-c3ae-4d30-8a87-d3775b0f5153.jpg
/kaggle/input/drone-bird-classification/drone_or_bird/birds/c90c58c2-cdcb-4656-8a44-2f27a46d7e21.jpg
/kaggle/input/drone-bird-classification/drone_or_bird/birds/714db76d-d36d-409a-b807-201c5e39ef88.jpg
/kaggle/input/drone-bird-classification/drone_or_bird/birds/0a5b4074-52c7-42d5-8b5e-54b4f93fc129.jpg
/kaggle/input/drone-bird-classification/drone_or_bird/birds/b8d33001-d21f-4ac9-8ed0-9523faf9ca3d.jpg
/kaggle/input/drone-bird-classification/drone_or_bird/birds/fa810b19-d525-4b80-aece-223c021c00ea.jpg
/kaggle/input/drone-bird-classification/drone_or_bird/birds/e9188796-d823-47c1-8007-75946116ea1e.jpg
/kaggle/input/drone-bird-classification/drone_or_bird/birds/e44b5602-9218-4789-8bfc-3907bf5

In [3]:
dircs1=os.listdir('/kaggle/input/drone-bird-classification/drone_or_bird/birds/')
dircs2=os.listdir('/kaggle/input/drone-bird-classification/drone_or_bird/drones/')

In [4]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from keras.models import Model

In [6]:
images1=[]
labels1=[]
for directory in dircs1:
    image_paths=os.listdir(f'/kaggle/input/drone-bird-classification/drone_or_bird/birds/')
    for img_name in image_paths:
        ext = os.path.splitext(img_name)[-1].lower()
        if ext.lower().endswith(('.png', '.jpg', '.jpeg')):
            images1.append(f'/kaggle/input/drone-bird-classification/drone_or_bird/birds/{img_name}')
            labels1.append("bird")

In [7]:
for directory in dircs1:
    image_paths=os.listdir(f'/kaggle/input/drone-bird-classification/drone_or_bird/drones/')
    for img_name in image_paths:
        ext = os.path.splitext(img_name)[-1].lower()
        if ext.lower().endswith(('.png', '.jpg', '.jpeg')):
            images1.append(f'/kaggle/input/drone-bird-classification/drone_or_bird/drones/{img_name}')
            labels1.append("drone")

In [8]:
df_dict={"images":images1,"labels":labels1}

In [9]:
df=pd.DataFrame(df_dict)

In [10]:
df.head(10)

,images,labels
0,/kaggle/input/drone-bird-classification/drone_...,bird
1,/kaggle/input/drone-bird-classification/drone_...,bird
2,/kaggle/input/drone-bird-classification/drone_...,bird
3,/kaggle/input/drone-bird-classification/drone_...,bird
4,/kaggle/input/drone-bird-classification/drone_...,bird
5,/kaggle/input/drone-bird-classification/drone_...,bird
6,/kaggle/input/drone-bird-classification/drone_...,bird
7,/kaggle/input/drone-bird-classification/drone_...,bird
8,/kaggle/input/drone-bird-classification/drone_...,bird
9,/kaggle/input/drone-bird-classification/drone_...,bird


In [11]:
df.tail(10)

,images,labels
53450,/kaggle/input/drone-bird-classification/drone_...,drone
53451,/kaggle/input/drone-bird-classification/drone_...,drone
53452,/kaggle/input/drone-bird-classification/drone_...,drone
53453,/kaggle/input/drone-bird-classification/drone_...,drone
53454,/kaggle/input/drone-bird-classification/drone_...,drone
53455,/kaggle/input/drone-bird-classification/drone_...,drone
53456,/kaggle/input/drone-bird-classification/drone_...,drone
53457,/kaggle/input/drone-bird-classification/drone_...,drone
53458,/kaggle/input/drone-bird-classification/drone_...,drone
53459,/kaggle/input/drone-bird-classification/drone_...,drone


In [12]:
onehot=OneHotEncoder()
lab=onehot.fit_transform(df['labels'].values.reshape(-1,1)).toarray()

In [13]:
images=df['images']

In [14]:
jpg=("/kaggle/input/drone-bird-classification/drone_or_bird/drones/108759dc-ad0c-4df0-95cf-818b30c30970.jpg")
image = cv2.imread(jpg)

In [15]:
r = 150.0 / image.shape[1]
dim = (150, int(image.shape[0] * r))

In [16]:
imgs=[]
for img in images:
    imagecv=cv2.imread(img)
    if imagecv is None:
        continue
    image=cv2.resize(imagecv,dim, interpolation= cv2.INTER_LINEAR)
    imgs.append(image)

In [17]:
images=np.array(imgs)
labels=np.array(lab)

In [18]:
x_train,x_test,y_train,y_test=train_test_split(images,labels,random_state=42,test_size=0.1)

In [19]:
aug = ImageDataGenerator(rotation_range=20,
zoom_range=0.15,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.15,
horizontal_flip=True,
fill_mode="nearest")

In [22]:
num_classes=2
model = VGG16(include_top=False,weights='imagenet', input_shape=(128, 128, 3))
for layer in model.layers:
    layer.trainable = False

flat1 = tf.keras.layers.Flatten()(model.output)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(flat1)
model = Model(inputs=model.inputs, outputs=output)

In [23]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
metrics=["accuracy"])
callback=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=5,
    restore_best_weights=True
)

In [ ]:
H=model.fit(
x=aug.flow(x_train, y_train, batch_size=32),
validation_data=(x_test, y_test),
    batch_size=16,
epochs=50,callbacks=[callback])

2022-09-17 13:07:14.078854: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
